In [26]:
!pip install -U keras-nlp
!pip install -U keras

  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
Using cached keras-3.3.3-py3-none-any.whl (1.1 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wu

In [27]:
import keras
import keras_nlp
import numpy as np

In [28]:
import os

os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [30]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[256000,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomNormalV2]

In [31]:
import csv

data = []
template = "Question:\n{question}\n\nAnswer:\n{answer}"

with open("/kaggle/input/layoutlm/medquad.csv", mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data.append(template.format(**row))

In [32]:
data[1]

'Question:\nWhat causes Glaucoma ?\n\nAnswer:\nNearly 2.7 million people have glaucoma, a leading cause of blindness in the United States. Although anyone can get glaucoma, some people are at higher risk. They include - African-Americans over age 40  - everyone over age 60, especially Hispanics/Latinos  - people with a family history of glaucoma. African-Americans over age 40 everyone over age 60, especially Hispanics/Latinos people with a family history of glaucoma.  In addition to age, eye pressure is a risk factor. Whether you develop glaucoma depends on the level of pressure your optic nerve can tolerate without being damaged. This level is different for each person. Thats why a comprehensive dilated eye exam is very important. It can help your eye care professional determine what level of eye pressure is normal for you. Another risk factor for optic nerve damage relates to blood pressure. Thus, it is important to also make sure that your blood pressure is at a proper level for you

In [33]:
prompt = template.format(
    question="What is (are) Diabetes ?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1715837384.052465      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1715837384.200377      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Question:
What is (are) Diabetes ?

Answer:
Diabetes is a disease in which the body does not make or use insulin properly. Insulin is a hormone that helps glucose (sugar) get into the cells of the body. When the body does not make or use insulin properly, glucose builds up in the blood. This is called hyperglycemia.


In [34]:
prompt = template.format(
    question="How to prevent Diabetes ?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
How to prevent Diabetes ?

Answer:
Diabetes is a disease that affects the way your body uses sugar (glucose). When you eat carbohydrates, your body breaks them down into sugar. Insulin is a hormone that helps your body use the sugar for energy. If you have diabetes, your body either doesn't make enough insulin or doesn't use it properly. This causes too much sugar to build up in your blood.


In [36]:
#gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.preprocessor.sequence_length = 128
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

W0000 00:00:1715837463.166285     113 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


16412/16412 ━━━━━━━━━━━━━━━━━━━━ 3171s 191ms/step - loss: 1.3632 - sparse_categorical_accuracy: 0.6572


In [45]:
prompt = template.format(
    question="What is (are) raju ?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
What is (are) raju ?

Answer:
The raju is a rare, inherited disorder that causes the body to produce too much of a protein called angiotensin-converting enzyme (ACE). ACE is a protein that helps control blood pressure. The excess ACE protein causes the blood vessels to narrow, which can lead to high blood pressure.
                
The raju is a genetic disorder. It is caused by a mutation in the ACE gene. The ACE gene is located on the long arm of chromosome 17. The mutation causes the ACE gene to produce an abnormal form of ACE. The abnormal ACE protein is not able to control blood pressure.


In [53]:
prompt = template.format(
    question="my diabites is 150",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
my diabites is 150

Answer:
The answer is yes.  
            
The American Diabetes Association (ADA) recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that people with diabetes eat a diet that is low in fat and high in fiber.  
            
The ADA recommends that 

In [39]:
gemma_lm.save("version_finetuned.keras")